In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

import gtsam
from gtsam import imuBias, noiseModel, PriorFactorConstantBias
from gtsam.symbol_shorthand import B, V, X, L

from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data
from lac.params import LAC_BASE_PATH, DT

from lac.slam.feature_tracker import FeatureTracker

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

# IMU preintegration

References:

- https://github.com/borglab/gtsam/blob/develop/python/gtsam/examples/ImuFactorExample.py
- https://github.com/borglab/gtsam/blob/develop/python/gtsam/examples/PreintegrationExample.py
- https://github.com/alextsolovikos/superpoint-gtsam-vio/blob/VIO/src/VisualInertialOdometry.py


In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(-g)
gyro_sigma = 1e-3
accel_sigma = 1e-3
# gyro_sigma = 0.5
# accel_sigma = 0.5
integration_sigma = 1e-7
# integration_sigma = 0.5
I_3x3 = np.eye(3)
IMU_PARAMS.setGyroscopeCovariance(gyro_sigma**2 * I_3x3)
IMU_PARAMS.setAccelerometerCovariance(accel_sigma**2 * I_3x3)
IMU_PARAMS.setIntegrationCovariance(integration_sigma**2 * I_3x3)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# Fix first pose
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[0])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[0]))

# Zero bias
BIAS_KEY = B(0)
zero_bias = imuBias.ConstantBias(np.zeros(3), np.zeros(3))
# graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, noiseModel.Constrained.All(6)))
BIAS_NOISE = gtsam.noiseModel.Isotropic.Sigma(6, 1e-2)
# BIAS_NOISE = gtsam.noiseModel.Isotropic.Sigma(6, 0.1)
graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, BIAS_NOISE))
initial_estimate.insert(BIAS_KEY, zero_bias)

# Zero initial velocity prior
VELOCITY_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, 1e-2)
# VELOCITY_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
graph.push_back(gtsam.PriorFactorVector(V(0), np.zeros(3), VELOCITY_NOISE))
initial_estimate.insert(V(0), np.zeros(3))

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

N_FRAMES = 1000
RATE = 10

i = 0

POSE_NOISE = gtsam.noiseModel.Isotropic.Sigma(6, 1e-2)  # Stronger prior

for k in range(1, N_FRAMES):
    accel = imu_data[k, :3]
    gyro = imu_data[k, 3:]
    accum.integrateMeasurement(accel, gyro, DT)

    if k % RATE == 0:
        i += 1

        # init pose estimate
        initial_estimate.insert(X(i), gtsam.Pose3(poses[k]))

        initial_estimate.insert(V(i), np.array([0.0, 0.0, 0]))
        # initial_velocity = (poses[i][:3,3] - poses[i-1][:3,3]) / DT
        # initial_estimate.insert(V(i), initial_velocity)

        # add pose prior
        graph.add(gtsam.PriorFactorPose3(X(i), gtsam.Pose3(poses[k]), POSE_NOISE))

        graph.add(gtsam.ImuFactor(X(i - 1), V(i - 1), X(i), V(i), BIAS_KEY, accum))

        accum.resetIntegration()

In [ ]:
params = gtsam.LevenbergMarquardtParams()

optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()

In [ ]:
# Get bias
bias = result.atConstantBias(BIAS_KEY)
print(f"Bias: {bias}")

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(i)])

fig = plot_poses(poses[:N_FRAMES], no_axes=True, color="black", name="Ground Truth")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM IMU")
fig.show()

# Stereo VO


In [ ]:
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.util import load_stereo_images

In [ ]:
# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)
assert len(left_imgs.keys()) == len(right_imgs.keys())
img_idxs = sorted(left_imgs.keys())

In [ ]:
svo = StereoVisualOdometry(cam_config)
START_FRAME = 80
svo.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
# Pre-process the VO
svo_poses = [initial_pose]
pose_deltas = []

END_FRAME = 4500

for idx in tqdm(np.arange(START_FRAME + 2, END_FRAME, 2)):
    svo.track(left_imgs[idx], right_imgs[idx])
    svo_poses.append(svo.rover_pose)
    pose_deltas.append(svo.pose_delta)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

svo_pose_sigma = 1e-2 * np.ones(6)
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(svo_pose_sigma)

values.insert(X(0), gtsam.Pose3(initial_pose))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(initial_pose)))

# Add odometry factors from VO
for i in range(1, len(svo_poses)):
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(
        gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(pose_deltas[i - 1]), svo_pose_noise)
    )

In [ ]:
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, values, params)
result = optimizer.optimize()

opt_poses = [result.atPose3(X(i)).matrix() for i in range(len(svo_poses))]

In [ ]:
fig = plot_poses(poses[:END_FRAME], no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="VO")
fig = plot_poses(opt_poses, no_axes=True, fig=fig, color="green", name="Optimized")
fig.show()

messing with visual-inertial


In [ ]:
orb = cv2.ORB_CREATE()


def detect_keypoints(img):
    frame = cv2.imread(str(data_path / "FrontLeft" / f"{img}.png"), cv2.IMREAD_GRAYSCALE)
    keypts = orb.detect(frame, None)
    keypts, desc = orb.compute(frame, keypts)
    pts = np.array([kp.pt for kp in keypts], dtype=np.float32)
    return pts, desc


def track_mono(img, prev_img, prev_pts):
    # track with optical flow
    prev_frame = cv2.imread(str(data_path / "FrontLeft" / f"{prev_img}.png"), cv2.IMREAD_GRAYSCALE)
    next_frame = cv2.imread(str(data_path / "FrontLeft" / f"{img}.png"), cv2.IMREAD_GRAYSCALE)
    next_keypts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, next_frame, prev_pts, None)

    # keep only valid points
    tracked_pts = next_keypts[status.flatten() == 1]
    return tracked_pts


def match_features_stereo(left_pts, left_desc, right_pts, right_desc):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(left_desc, right_desc)
    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = []
    for m in matches:
        if m.distance < 30:
            good_matches.append(m)
    left_pts = np.array([left_pts[m.queryIdx] for m in good_matches])
    right_pts = np.array([right_pts[m.trainIdx] for m in good_matches])
    return left_pts, right_pts


def triangulate_pts_stereo(left_pts, right_pts, K, baseline):
    # left_pts_h = np.hstack((left_pts, np.ones((left_pts.shape[0], 1))))
    # right_pts_h = np.hstack((right_pts, np.ones((right_pts.shape[0], 1))))

    disparity = left_pts[:, 0] - right_pts[:, 0]
    disparity[disparity == 0] = 1e-6

    Z = K[0, 0] * baseline / disparity
    X = (left_pts[:, 0] - K[0, 2]) * Z / K[0, 0]
    Y = (left_pts[:, 1] - K[1, 2]) * Z / K[1, 1]
    pts3d = np.vstack((X, Y, Z)).T

    return pts3d

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# camera noise model
POSE_NOISE = gtsam.noiseModel.Isotropic.Sigma(6, 1e-3)
LANDMARK_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, 1e-2)

# add first pose
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[0])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[0]))

# add initial landmarks
init_keypts, init_desc = detect_keypoints(img_idxs[0])
tracked_pts = track_mono(img_idxs[1], img_idxs[0], init_keypts)
init_landmarks = triangulate_pts(init_keypts, tracked_pts, cam_config.K, np.eye(3), np.zeros(3))

for i, pt in enumerate(init_landmarks):
    graph.add(gtsam.PriorFactorPoint3(L(i), pt.squeeze(), LANDMARK_NOISE))
    initial_estimate.insert(L(i), pt.squeeze())
